<a href="https://colab.research.google.com/github/cc1528/MedSAM/blob/main/peft_llama2_skeleton_rev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -q -U peft transformers datasets==2.15.0 bitsandbytes trl==0.7.4 accelerate wandb

In [2]:
#pip install --upgrade trl

Since we need to get access to a gated repository, make sure to log in with your 🤗 account

In [4]:
from huggingface_hub import notebook_login

notebook_login()


In [5]:
from peft import LoraConfig

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Load the 7b llama model
model_id = "meta-llama/Llama-2-7b-hf"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
  )

# Load model
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.

tokenizer.add_special_tokens({'pad_token': '<PAD>'})  #

model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32001, 4096)

In [7]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096

In [25]:
model = prepare_model_for_kbit_training(model)

In [27]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# Method 1
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# Method 2
#model.add_adapter(lora_config)

trainable params: 19,988,480 || all params: 6,758,412,288 || trainable%: 0.2958


In [28]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PeftModelForCausalLM(
          (base_model): LoraModel(
            (model): LlamaForCausalLM(
              (model): LlamaModel(
                (embed_tokens): Embedding(32001, 4096)
                (layers): ModuleList(
                  (0-31): 32 x LlamaDecoderLayer(
                    (self_attn): LlamaAttention(
                      (q_proj): lora.Linear4bit(
                        (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                        (lora_dropout): ModuleDict(
                          (default): Identity()
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=4096, out_features=8, bias=False)
                        )
                        (lora_B): ModuleDict(
                          (default): Linear(in_features=8, out_f

In [29]:
from datasets import load_dataset

train_dataset = load_dataset("stingning/ultrachat", split="train[:1%]")

In [30]:
from transformers import TrainingArguments

YOUR_HF_USERNAME = "CinthyaCriollo"

output_dir = f"{YOUR_HF_USERNAME}/llama-7b-qlora-ultrachat"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)


In [33]:
from huggingface_hub import notebook_login

notebook_login()


In [34]:
from trl import SFTTrainer

def formatting_func(example):
    text = f"### USER: {example['data'][0]}\n### ASSISTANT: {example['data'][1]}"
    return text

In [24]:
#help(SFTTrainer)


In [35]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    #packing=True,
    #dataset_text_field="id",
    #tokenizer=tokenizer,

    #max_seq_length=1024,
    formatting_func=formatting_func,
    processing_class=tokenizer
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [36]:
trainer.train()

Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000


KeyboardInterrupt: 

In [37]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

model_id = "ybelkada/llama-7b-qlora-ultrachat"

tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    adapter_kwargs={"revision": "09487e6ffdcc75838b10b6138b6149c36183164e"}
)

text = "### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant:"

inputs = tokenizer(text, return_tensors="pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("After attaching Lora adapters:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

tokenizer_config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


adapter_config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

...

model-00014-of-00014.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

After attaching Lora adapters:
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant: Contrastive learning in machine learning is a technique that involves training a model to learn from data that is similar to the target data. ### USER: How does contrastive learning work in machine learning?### Assistant: In contrastive learning, the model is trained to learn from data that is similar to the target data. This is done by using a loss function that penalizes the model for making mistakes on the target data. The model is then trained to minimize this loss function, which helps it learn from the similar data.</s>


In [38]:
model.disable_adapters()
outputs = model.generate(inputs.input_ids, max_new_tokens=250, do_sample=False)

print("Before Lora:")
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Before Lora:
<s> ### USER: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant: Sure. Unterscheidung: Kontrastive Lernen ist ein Lernverfahren, bei dem ein Modell aus einem Datenbestand heraus trainiert wird. Der Datenbestand besteht aus zwei oder mehreren Datensätzen, die sich in einem oder mehreren Merkmalen unterscheiden. Der Merkmalsraum ist also nicht homogen. Das bedeutet, dass die Daten nicht nur unterschiedliche Werte annehmen können, sondern auch unterschiedliche Merkmale. Das Kontrastive Lernen ist ein Lernverfahren, bei dem ein Modell aus einem Datenbestand heraus trainiert wird. Der Datenbestand besteht aus zwei oder mehreren Datensätzen, die sich in einem oder mehreren Merkmalen unterscheiden. Der Merkmalsraum ist also nicht homogen. Das bedeutet, dass die Daten nicht nur unterschiedliche Werte annehmen können, sondern auch unterschiedliche Merkmale. Das Kontrastive Lernen ist ein Lernverfahren, bei dem 